In [53]:
import boto3
from datetime import date
import pandas as pd

In [54]:
today = date.today()
#today_str = today.strftime('%Y-%m-%d')
today_str = '2025-09-24'

s3 = boto3.client('s3', region_name='ap-northeast-2')

bucket_name = 'real-estate-avm'
object_key = f'processed/preprocessed_apt_txn/dt={today_str}/{today_str}.csv'
local_file = 'data/file.csv'

obj = s3.get_object(Bucket=bucket_name, Key=object_key)
df = pd.read_csv(obj['Body'])

df.head()

,단지명,전용면적(㎡),층,건축년도,도로명,면적당 단가(만원),아파트 나이,계약일자,경도,위도
0,강남데시앙포레,84.92,4,2014,광평로34길 55,2425.812529,11,2025-09-24,127.092172,37.480856


In [55]:
# === 필요한 라이브러리 불러오기 ===
import ee  # Google Earth Engine 파이썬 API
import requests  # 웹 요청 (썸네일 이미지 다운로드)
import pandas as pd
from PIL import Image  # 이미지 파일 열고 저장
from io import BytesIO  # 이미지 바이트 데이터를 PIL로 읽기 위한 버퍼
from datetime import datetime, timedelta, date  # 날짜 처리용
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from dotenv import load_dotenv
import os 

import logging
from logging.handlers import RotatingFileHandler

In [56]:
project = 'aptprice-464102'

In [57]:
# === Earth Engine 초기화 ===
ee.Authenticate()
ee.Initialize(project=project)
print("Google Earth Engine 초기화 완료")

Google Earth Engine 초기화 완료


In [58]:

# --- 1. 데이터프레임 순회 ---
# DataFrame의 각 행을 순회하며 위성 이미지를 가져옵니다.
for idx, row in df.iterrows():
    
    # --- 2. 이미지 검색을 위한 정보 설정 ---
    lat = row['위도']
    lon = row['경도']
    tx_date = datetime.strptime(row['계약일자'], "%Y-%m-%d")
    
    # 검색할 날짜 범위 설정 (계약일 기준 앞뒤 6개월)
    start_date = (tx_date - timedelta(days=183)).strftime('%Y-%m-%d')
    end_date = (tx_date + timedelta(days=183)).strftime('%Y-%m-%d')

    # 검색할 지역(ROI, Region of Interest) 설정 (중심점에서 1500m 반경)
    center = ee.Geometry.Point([lon, lat])
    roi = center.buffer(1500).bounds()

    print(f"\n[{idx}] 처리 중: {row['계약일자']} / lat={lat:.4f}, lon={lon:.4f}")

    # --- 3. Earth Engine에서 이미지 컬렉션 검색 ---
    # 구름이 5% 미만인 Sentinel-2 위성 이미지 검색
    collection = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(center)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
    )

    # 검색된 이미지가 없으면 건너뛰기
    if collection.size().getInfo() == 0:
        print(f" -> [실패] 해당 조건의 위성 이미지를 찾을 수 없습니다.")
        continue

    # 가장 최신 이미지를 선택
    image = collection.sort('system:time_start', False).first()

    # --- 4. 이미지 시각화 파라미터 설정 ---
    # 이미지의 밝기 등을 조절하기 위해 픽셀 값의 최소/최대 범위를 동적으로 계산
    stats = image.reduceRegion(
        reducer=ee.Reducer.percentile([2, 98]),
        geometry=roi, scale=10, maxPixels=1e8
    ).getInfo()

    if not stats:
        print(f" -> [실패] 이미지 통계 정보를 계산할 수 없습니다.")
        continue
        
    vis_params = {
        'region': roi,
        'format': 'jpg',
        'bands': ['B4', 'B3', 'B2'], # RGB 밴드
        'min': [stats.get('B4_p2', 500), stats.get('B3_p2', 500), stats.get('B2_p2', 500)],
        'max': [stats.get('B4_p98', 3500), stats.get('B3_p98', 3500), stats.get('B2_p98', 3500)],
        'scale': 10
    }

    # --- 5. 이미지 다운로드 및 저장 ---
    try:
        # 이미지 다운로드 URL 생성
        url = image.getThumbURL(vis_params)
        
        # URL로부터 이미지 데이터 요청
        response = requests.get(url)
        response.raise_for_status() # 요청 실패 시 에러 발생
        
        # 이미지 데이터를 PIL Image 객체로 변환
        img = Image.open(BytesIO(response.content))
        
        # 파일로 저장
        img.save(f"data/apt_image_{idx}.jpg")
        print(f" -> [성공] apt_image_{idx}.jpg 로 저장 완료.")

    except Exception as e:
        print(f" -> [실패] 이미지 다운로드 또는 저장 중 예외 발생: {e}")

print("\n--- 이미지 다운로드 완료 ---")


[0] 처리 중: 2025-09-24 / lat=37.4809, lon=127.0922
 -> [성공] apt_image_0.jpg 로 저장 완료.

--- 이미지 다운로드 완료 ---
